# TFRcord 格式
TFRecord 是Google官方推荐的一种数据格式，是Google专门为TensorFlow设计的一种数据格式。

实际上，TFRecord是一种二进制文件，其能更好的利用内存，其内部包含了多个tf.train.Example， 而Example是protocol buffer

(protocol buffer是google的一个开源项目,它是用于结构化数据串行化的灵活、高效、自动的方法，例如XML，不过它比xml更小、更快、也更简单。一种语言无关、平台无关、扩展性好的用于通信协议、数据存储的结构化数据序列化方法。) 

数据标准的实现，在一个Example消息体中包含了一系列的tf.train.feature属性，而 每一个feature 是一个key-value的键值对，其中，key 是string类型，而value 的取值有三种：
- bytes_list: 可以存储string 和byte两种数据类型。
- float_list: 可以存储float(float32)与double(float64) 两种数据类型 。
- int64_list: 可以存储：bool, enum, int32, uint32, int64, uint64 。

值的一提的是，TensorFlow 源码中到处可见.proto 的文件，且这些文件定义了TensorFlow重要的数据结构部分，且多种语言可直接使用这类数据，很强大。

# 为什么使用TFRecord
TFRecord 并非是TensorFlow唯一支持的数据格式，也可以使用CSV或文本等格式，但是对于TensorFlow来说，TFRecord 是最友好的，也是最方便的。
Google官方推荐在对于中大数据集来说，先将数据集转化为TFRecord数据（.tfrecords), 这样可加快你在数据读取, 预处理中的速度。

我们以cifar-10举例。

生成TFrecord只需要三步：

In [1]:
# 具体脚本请参照github
def main(img_path, label_path, tfrecords):
    # step 1. 定义一个writer对象
    writer = tf.python_io.TFRecordWriter(tfrecords)

    # Load npy data.
    imgs = np.load(img_path)
    labels = np.load(label_path)

    for _ in range(imgs.shape[0]):

        img = imgs[_]
        label = labels[_]

        height = img.shape[0]
        width = img.shape[1]

        img_raw = img.tostring()
        label_raw = label.tostring()

        # step 2. 定义features
        example = tf.train.Example(features=tf.train.Features(feature={
            'height': _int64_feature(height),
            'width': _int64_feature(width),
            'image_raw': _bytes_feature(img_raw),
            'mask_raw': _bytes_feature(label_raw)}))

        # step 3. 序列化写入
        writer.write(example.SerializeToString())

    writer.close()